In [173]:
import pandas as pd
import os
import csv
import numpy as np

In [ ]:
#dataframe of all respiratory deaths at the state level
state_data = pd.read_csv('Underlying Cause of Death, 1999-2017_allyears_states.txt', sep='\t')

In [330]:
#Dataframe of state populations from 2000-2010
state_pops_2000 = pd.read_csv('co-est00int-tot.csv', encoding='latin-1')
#Dataframe of state populations from 2010-2018
state_pops_2010 = pd.read_csv('co-est2018-alldata.csv', encoding='latin-1')

In [266]:
#counties_data = pd.read_csv('county_data/' + 'Underlying Cause of Death, 1999-2017_2001_counties.txt', sep='\t')

counties_data = pd.DataFrame()
for file in os.listdir('county_data'):
    #if file != 'Underlying Cause of Death, 1999-2017_2012_counties.txt':
    new_year = pd.read_csv('county_data/' + file, sep='\t', na_filter = False)
    counties_data = pd.concat([counties_data, new_year], axis=0)


/Users/Marta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [267]:
col_list = ['Notes', 'Population', 'Crude Rate']
for name in col_list:
    counties_data = counties_data.drop(columns = name)



In [268]:
def split_column_names(dataframe, column1, column2, column3, splitter):
    '''This function creates a new dataframe that has one specified column from the original dataframe.
    Then it adds two new columns to the original dataframe (or replaces one of the original columns if the names are 
    the same) with the split values.
    
    The column name inputs must be strings.
    The splitter is the symbol or letter that is used to split the string.
    '''
    
    new_df = dataframe[column1].str.split(splitter, n=1, expand=True)
    dataframe[column2] = new_df[0]
    dataframe[column3] = new_df[1]
    
    return dataframe
    

In [269]:
counties_data = split_column_names(counties_data, 'County', 'County', 'State Abr', ',')

In [271]:
counties_data = split_column_names(counties_data, 'Month Code', 'Year', 'Month', '/')

In [138]:
#Making a dataframe of the states and their abbreviations
state_abrevs = pd.read_csv('state_abrevs.csv')

In [141]:
#Renaming columns in the state abbrevs dataframe so that they match existing column names in the previous dataframes
state_abrevs = state_abrevs.rename(columns={"State": "STNAME", 'State Abrev': 'State'})

In [155]:
#Merging the state abrevs with the county population data for 2010
state_pops_2010_mergetest = pd.merge(state_pops_2010[['STNAME', 'CTYNAME', 'CENSUS2010POP',
       'ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018']], state_abrevs, on='STNAME')

In [213]:
#Merging the state abrevs with the county population data for 2000
state_pops_2000_mergetest = pd.merge(state_pops_2000[['STNAME', 'CTYNAME',
       'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009']], state_abrevs, on='STNAME')

In [284]:
def changing_col_name(dataframe, name_before, name_after):
    '''This function changes the name of one column and returns the dataframe.
    '''
    
    return dataframe.rename(columns={name_before: name_after})

In [331]:
state_pops_2010 = changing_col_name(state_pops_2010, 'CTYNAME', 'County')
state_pops_2000 = changing_col_name(state_pops_2000, 'CTYNAME', 'County')
state_pops_2010 = changing_col_name(state_pops_2010, 'STNAME', 'State')
state_pops_2000 = changing_col_name(state_pops_2000, 'STNAME', 'State')

In [332]:
#Merging county population data for 2010 with county death rates
county_pop_merge = pd.merge(state_pops_2010, counties_data, on=['County','State'])


In [307]:
#Merging county population data for 2000 with county death rates
county_pop_merge_2000 = pd.merge(state_pops_2000, counties_data, on=['County', 'State'])


In [228]:
#Checking to see if it matched the wrong county names anywhere
#Need to work on this because right now it's not recognizing that the two columns have the same values in them
county_pop_merge['que'] = np.where((county_pop_merge['State_x']==county_pop_merge['State_y']), county_pop_merge['State_y'], np.nan)

In [244]:
#Checking to see if it matched the wrong county names anywhere
#Need to work on this because right now it's not recognizing that the two columns have the same values in them
county_pop_merge_2000['que'] = np.where((county_pop_merge_2000['State_x']==county_pop_merge_2000['State_y']), county_pop_merge_2000['State_y'], np.nan)

In [342]:
def choose_data_by_year(dataframe, year):
    '''This function takes a dataframe as an input and the target year as an integer, and outputs a dataframe that contains 
    only that specific year.   
    
    It also specifies which columns we want to keep in the final dataframe, and it renames the population column
    so that it does not have a year-specific name.
    '''
    return dataframe[['County',
       'POPESTIMATE'+str(year),
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State', 'Year']].loc[dataframe["Year"]==str(year)].rename(columns={'POPESTIMATE'+str(year):'Population'})
    
    
    


In [319]:
def concat_dfs_vertically(dataframe1, dataframe2):
    '''This function merges two dataframes vertically.
    It assumes they have the same column names already.
    '''
    return pd.concat([dataframe1, dataframe2])
    
    

In [343]:
#Getting all the data for each year in individual dataframes and then combining the new data frames

all_years = pd.DataFrame()
for year in range(2010, 2018):
    new_df = choose_data_by_year(county_pop_merge, year)
    all_years = concat_dfs_vertically(all_years, new_df)
    
for year in range(2000, 2010):
    new_df = choose_data_by_year(county_pop_merge_2000, year)
    all_years = concat_dfs_vertically(all_years, new_df)

In [349]:
type(all_years['Population'][0])

numpy.int64

In [351]:
#Calculating the % of total deaths
all_years['% of Total Deaths'] = np.int64(all_years['Deaths'])/all_years['Population']

In [354]:
#Creating a new column that changes the units to make them easier to deal with 
all_years['% of Total Deaths (x10^6)'] = all_years['% of Total Deaths'] * 1000000

In [ ]:
county_pop_2010 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2010', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2010']
county_pop_2011 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2011', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2011']
county_pop_2012 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2012', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2012']
county_pop_2013 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2013', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2013']
county_pop_2014 = county_pop_merge[['STNAME', 'County',
       'POPESTIMATE2014', 'State_x',
       '% of Total Deaths', 'County Code', 'Deaths', 'Month', 'Month Code',
       'State_y', 'Year', 'que']].loc[county_pop_merge["Year"]=='2014']

In [ ]:
#Renaming the population column
county_pop_2010 = county_pop_2010.rename(columns={'POPESTIMATE2010':'Population'})
county_pop_2011 = county_pop_2011.rename(columns={'POPESTIMATE2011':'Population'})
county_pop_2012 = county_pop_2012.rename(columns={'POPESTIMATE2012':'Population'})

In [ ]:
new = counties_data["County"].str.split(",", n = 1, expand = True)

In [ ]:
counties_data["County"]= new[0]
counties_data['State'] = new[1]

In [ ]:
new_date = counties_data['Month Code'].str.split('/', n=1, expand=True)

In [ ]:
counties_data['Year']=new_date[0]
counties_data['Month']=new_date[1]

In [ ]:
#Changing the name of the CTYNAME column in the population data 2000
state_pops_2000_mergetest = state_pops_2000_mergetest.rename(columns={'CTYNAME':'County'})

In [ ]:
#Changing the name of the CTYNAME column in the population data 2010
state_pops_2010_mergetest = state_pops_2010_mergetest.rename(columns={'CTYNAME':'County'})